In [1]:
# ======================================================================================
# COLAB ENVIRONMENT SETUP (Fixed for Oct 2025)
# ======================================================================================

print("⏳ Installing all required Python packages... This may take a few minutes.")

# --- Full environment setup with binary compatibility fix ---
!pip install -U numpy==1.26.4 scikit-learn==1.3.2 --force-reinstall --no-cache-dir
!pip install transformers==4.38.2 datasets==2.18.0 sentence-transformers==2.7.0 \
rouge-score==0.1.2 bert-score==0.3.13 networkx==3.2.1 hdbscan==0.8.33 \
nltk==3.8.1 torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 \
matplotlib seaborn umap-learn --extra-index-url https://download.pytorch.org/whl/cu118

print("✅ Python packages installed successfully and compatible with NumPy ABI.")

# --- Download NLTK resources ---
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # Some newer NLTK versions also need this
print("✅ Punkt tokenizer data reinstalled successfully!")

print("\n" + "="*80)
print("  ✅ SETUP COMPLETE. THE ENVIRONMENT IS NOW PREPARED.")
print("  ⚠️ IMPORTANT: Please restart the runtime before running your experiments.")
print("  Go to the menu: 'Runtime' -> 'Restart Session' (or 'Factory reset runtime').")
print("="*80)


⏳ Installing all required Python packages... This may take a few minutes.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 262.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 149.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 322.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 284.6 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uninstalled threadpoolctl-3.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.2
    Uninstalling joblib-1.5.2:
     

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# ======================================================================================
# HTAS-V3 — Multi-Dataset Colab Script (CNN + Multi-News + WikiSum mix, 10k samples)
# - Uses Sentence-BERT embeddings + optional UMAP + HDBSCAN clustering
# - Mixes datasets to `config.total_train_samples` according to config.ratios
# - Fine-tunes HTAS (BART-base), evaluates baselines, saves metrics/plots
# ======================================================================================

import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import time
import gc
import json
import random
from pathlib import Path
from collections import defaultdict
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset as TorchDataset

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    get_linear_schedule_with_warmup,
    DataCollatorForSeq2Seq,
    logging as hf_logging
)
from rouge_score import rouge_scorer
from bert_score import score as bert_score_calculator

import nltk
from nltk.tokenize import sent_tokenize

# Optional metrics
try:
    import sacrebleu
    _HAS_SACREBLEU = True
except Exception:
    _HAS_SACREBLEU = False

try:
    from nltk.translate.meteor_score import meteor_score
    _HAS_METEOR = True
except Exception:
    _HAS_METEOR = False

# Clustering helpers
import hdbscan
import networkx as nx

# Optional UMAP
try:
    import umap.umap_ as umap
    _HAS_UMAP = True
except Exception:
    _HAS_UMAP = False

hf_logging.set_verbosity_error()
sns.set_style("whitegrid")

# --------------------------
# Configuration
# --------------------------
class Config:
    """
    Central configuration for dataset mixing, model, and runtime behavior.
    """
    # Dataset mixing targets
    total_train_samples = 8000   # total combined training pool
    total_test_samples = 2000    # total combined test pool

    # Dataset ratio (must sum to 1.0)
    # Updated: replaced 'wikisum' with 'wikihow' for better open availability
    ratios = {
        'cnn_dailymail::3.0.0': 0.35,   # concise news summaries
        'multi_news': 0.35,            # multi-document news
        'xsum': 0.30                # procedural + long-form knowledge
    }

    # Model
    htas_model_name = 'facebook/bart-base'
    max_input_length = 1024
    max_target_length = 150

    # Baselines (zero-shot)
    eval_models = [
        'facebook/bart-large-cnn',
        't5-base',
        'google/pegasus-cnn_dailymail'
    ]

    # Training
    batch_size = 4
    gradient_accumulation_steps = 2
    learning_rate = 3e-5
    max_steps = 2000
    epochs = 3
    warmup_ratio = 0.1
    fp16 = True

    # HTAS
    token_budget = 400
    min_cluster_size = 3
    per_cluster_topk = 2
    fallback_topk = 8
    sim_threshold = 0.15

    # PageRank
    pagerank_alpha = 0.85
    pagerank_max_iter = 100

    # Evaluation
    num_beams = 4
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    seed = 42

    # Output
    output_dir = Path('/content/htas_output')

config = Config()

# --------------------------
# Utilities
# --------------------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def mkdir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# Ensure NLTK punkt is available
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# --------------------------
# HTAS components (Sentence embeddings, optional UMAP, HDBSCAN)
# --------------------------
class SentenceEncoderGPU:
    def __init__(self, device: str = config.device):
        self.device = device
        # small & fast SBERT for sentence embeddings
        self.model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        self.model.eval()

    def encode(self, sentences: List[str]) -> torch.Tensor:
        if not sentences:
            return torch.empty(0, 384, device=self.device)
        return self.model.encode(sentences, convert_to_tensor=True, device=self.device, show_progress_bar=False)

    def cosine_similarity(self, a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        a_norm = F.normalize(a, p=2, dim=-1)
        b_norm = F.normalize(b, p=2, dim=-1)
        return torch.mm(a_norm, b_norm.t())

class HTASClusterer:
    def __init__(self, cfg):
        self.cfg = cfg

    def cluster_sentences(self, embeddings) -> np.ndarray:
        """
        embeddings: either torch.Tensor (N x D) or numpy array (N x D)
        returns labels np.ndarray shape (N,)
        """
        if isinstance(embeddings, torch.Tensor):
            n = embeddings.size(0)
            emb_cpu = embeddings.cpu().numpy()
        else:
            emb_cpu = np.asarray(embeddings)
            n = emb_cpu.shape[0]

        if n == 0:
            return np.array([], dtype=int)
        if n < self.cfg.min_cluster_size:
            return np.zeros(n, dtype=int)

        clusterer = hdbscan.HDBSCAN(min_cluster_size=self.cfg.min_cluster_size, min_samples=1, metric='euclidean')
        labels = clusterer.fit_predict(emb_cpu)

        # handle noise: assign to nearest non-noise cluster
        noise_mask = labels == -1
        if noise_mask.any() and (~noise_mask).any():
            non_noise_idx = np.where(~noise_mask)[0]
            for idx in np.where(noise_mask)[0]:
                # cosine similarity against non-noise
                vec = torch.tensor(emb_cpu[idx]).unsqueeze(0)
                non_noise_emb = torch.tensor(emb_cpu[non_noise_idx])
                sims = F.cosine_similarity(vec, non_noise_emb, dim=-1)
                labels[idx] = int(labels[non_noise_idx][sims.argmax().item()])

        return labels

    def select_from_cluster(self, cluster_indices: List[int], sim_matrix: torch.Tensor) -> List[int]:
        if not cluster_indices:
            return []
        if len(cluster_indices) <= self.cfg.per_cluster_topk:
            return cluster_indices

        sim_sub = sim_matrix[cluster_indices][:, cluster_indices].cpu().numpy()
        sim_sub[sim_sub < self.cfg.sim_threshold] = 0
        np.fill_diagonal(sim_sub, 0)
        try:
            G = nx.from_numpy_array(sim_sub)
            prs = nx.pagerank(G, alpha=self.cfg.pagerank_alpha, max_iter=self.cfg.pagerank_max_iter)
            deg = nx.degree_centrality(G)
            combined = {i: 0.6 * prs[i] + 0.4 * deg[i] for i in range(len(cluster_indices))}
            ranked = sorted(combined.items(), key=lambda x: x[1], reverse=True)
            return [cluster_indices[i] for i, _ in ranked[:self.cfg.per_cluster_topk]]
        except Exception:
            sums = sim_sub.sum(axis=1)
            idxs = np.argsort(-sums)[:self.cfg.per_cluster_topk]
            return [cluster_indices[int(i)] for i in idxs]

class TokenBudgetedSelector:
    def __init__(self, tokenizer, cfg):
        self.tokenizer = tokenizer
        self.cfg = cfg

    def select_with_budget(self, sentences: List[str], candidate_indices: List[int]) -> List[int]:
        selected = []
        total_tokens = 0
        for idx in candidate_indices:
            tokens = len(self.tokenizer.encode(sentences[idx], add_special_tokens=False))
            if total_tokens + tokens <= self.cfg.token_budget:
                selected.append(idx)
                total_tokens += tokens
            if total_tokens >= self.cfg.token_budget:
                break
        return selected

class HTASProcessor:
    def __init__(self, cfg, tokenizer):
        self.cfg = cfg
        self.tokenizer = tokenizer
        self.encoder = SentenceEncoderGPU(device=cfg.device)
        self.clusterer = HTASClusterer(cfg)
        self.selector = TokenBudgetedSelector(tokenizer, cfg)
        # UMAP reducer cached per processor to avoid re-init overhead
        self._umap_reducer = None
        if _HAS_UMAP:
            # reduce to 16 dims for clustering (configurable)
            try:
                self._umap_reducer = umap.UMAP(n_components=16, random_state=cfg.seed)
            except Exception:
                self._umap_reducer = None

    def preprocess_documents(self, documents: List[str]) -> List[str]:
        sentences = []
        for doc in documents:
            sents = sent_tokenize(doc)
            sentences.extend([s.strip() for s in sents if 8 < len(s.split()) < 150])
        return sentences

    def create_guided_input(self, documents: List[str]) -> str:
        sentences = self.preprocess_documents(documents)
        if not sentences:
            return " ".join(documents)

        embeddings = self.encoder.encode(sentences)  # tensor on device

        if len(embeddings) == 0:
            return " ".join(documents)

        # Similarity matrix computed on original embeddings (SBERT high-dim)
        sim_matrix = self.encoder.cosine_similarity(embeddings, embeddings)

        # For clustering we use UMAP-reduced vectors if available (faster and denoised)
        emb_cpu = embeddings.cpu().numpy()
        cluster_input = emb_cpu
        if self._umap_reducer is not None:
            try:
                reduced = self._umap_reducer.fit_transform(emb_cpu)
                cluster_input = reduced
            except Exception:
                cluster_input = emb_cpu

        labels = self.clusterer.cluster_sentences(cluster_input)
        selected_indices = []
        for label in np.unique(labels):
            cluster_indices = np.where(labels == label)[0].tolist()
            selected = self.clusterer.select_from_cluster(cluster_indices, sim_matrix)
            selected_indices.extend(selected)

        # fallback global PageRank if too few selected
        if len(selected_indices) < self.cfg.fallback_topk:
            sim_np = sim_matrix.cpu().numpy()
            sim_np[sim_np < self.cfg.sim_threshold] = 0
            np.fill_diagonal(sim_np, 0)
            try:
                G = nx.from_numpy_array(sim_np)
                pr = nx.pagerank(G, alpha=self.cfg.pagerank_alpha, max_iter=self.cfg.pagerank_max_iter)
                ranked = sorted(pr.items(), key=lambda x: x[1], reverse=True)
                for idx, _ in ranked:
                    if idx not in selected_indices:
                        selected_indices.append(idx)
                    if len(selected_indices) >= self.cfg.fallback_topk:
                        break
            except Exception:
                # simple fallback: take top similarity sums
                sums = sim_np.sum(axis=1)
                idxs = np.argsort(-sums)[:self.cfg.fallback_topk]
                for idx in idxs:
                    if idx not in selected_indices:
                        selected_indices.append(int(idx))

        final_indices = self.selector.select_with_budget(sentences, selected_indices)
        final_indices = sorted(set(final_indices))
        guidance = " ".join([sentences[i] for i in final_indices])
        full_text = " ".join(documents)
        return f"{guidance}</s><s>{full_text}"

# --------------------------
# Dataset & DataLoader
# --------------------------
class SummarizationDataset(TorchDataset):
    def __init__(self, texts: List[str], summaries: List[str], tokenizer, cfg):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.cfg = cfg

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        model_inputs = self.tokenizer(str(self.texts[idx]), max_length=self.cfg.max_input_length, truncation=True, padding=False)
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(str(self.summaries[idx]), max_length=self.cfg.max_target_length, truncation=True, padding=False)
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

# --------------------------
# Training loop (manual, fp16 supported)
# --------------------------
def train_model(model, tokenizer, train_dataset, cfg):
    model.to(cfg.device)
    model.train()
    collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
    dataloader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, collate_fn=collator)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate, eps=1e-8)

    num_steps = min(cfg.max_steps, (len(dataloader) // cfg.gradient_accumulation_steps) * cfg.epochs)
    num_warmup = int(cfg.warmup_ratio * num_steps) if num_steps > 0 else 0
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup, num_steps) if num_steps > 0 else None
    scaler = torch.cuda.amp.GradScaler(enabled=cfg.fp16 and cfg.device.startswith('cuda'))

    global_step = 0
    losses = []
    t0 = time.time()
    print(f"\n🚂 Training for {num_steps} steps...")

    for epoch in range(cfg.epochs):
        if global_step >= num_steps:
            break
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{cfg.epochs}")
        for step, batch in enumerate(pbar):
            if global_step >= num_steps:
                break
            batch = {k: v.to(cfg.device) for k, v in batch.items()}
            with torch.cuda.amp.autocast(enabled=cfg.fp16 and cfg.device.startswith('cuda')):
                outputs = model(**batch)
                loss = outputs.loss / cfg.gradient_accumulation_steps
            scaler.scale(loss).backward()
            if (step + 1) % cfg.gradient_accumulation_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                if scheduler:
                    scheduler.step()
                optimizer.zero_grad()
                global_step += 1
                val = float(loss.item() * cfg.gradient_accumulation_steps)
                losses.append(val)
                pbar.set_postfix({'loss': f'{val:.4f}'})

    elapsed = time.time() - t0
    print(f"✓ Fine-tuning finished in {elapsed/60:.2f} min")
    return model, losses

# --------------------------
# Metrics helpers
# --------------------------
def compute_bleu(reference: str, candidate: str) -> float:
    if _HAS_SACREBLEU:
        try:
            return float(sacrebleu.sentence_bleu(candidate, [reference]).score)
        except Exception:
            return 0.0
    else:
        try:
            ref_tokens = reference.split()
            cand_tokens = candidate.split()
            if not cand_tokens or not ref_tokens:
                return 0.0
            overlap = len(set(ref_tokens) & set(cand_tokens)) / max(1, len(set(ref_tokens)))
            return overlap * 100.0
        except Exception:
            return 0.0

def compute_meteor(reference: str, candidate: str) -> float:
    if _HAS_METEOR:
        try:
            return meteor_score([reference], candidate) * 100.0
        except Exception:
            return 0.0
    return 0.0

def compute_coverage_compression(source: str, summary: str, tokenizer) -> Tuple[float, float]:
    src_tokens = len(tokenizer.encode(source, add_special_tokens=False))
    sum_tokens = len(tokenizer.encode(summary, add_special_tokens=False))
    if src_tokens == 0:
        return 0.0, 0.0
    coverage = (sum_tokens / src_tokens) * 100.0
    compression = src_tokens / max(sum_tokens, 1)
    return coverage, compression

# --------------------------
# Batched generation & evaluation
# --------------------------
def batch_generate(model, tokenizer, inputs: List[str], cfg, batch_size=8):
    model.to(cfg.device)
    model.eval()
    outputs = []
    for i in range(0, len(inputs), batch_size):
        batch_texts = inputs[i:i+batch_size]
        enc = tokenizer(batch_texts, max_length=cfg.max_input_length, truncation=True, padding=True, return_tensors='pt').to(cfg.device)
        with torch.no_grad():
            summary_ids = model.generate(enc.input_ids, attention_mask=enc.attention_mask, num_beams=cfg.num_beams, max_length=cfg.max_target_length, min_length=30, length_penalty=1.0, early_stopping=True)
        for ids in summary_ids:
            outputs.append(tokenizer.decode(ids, skip_special_tokens=True).strip())
    return outputs

def evaluate_model_full(name: str, model, tokenizer, test_dataset, cfg, processor=None):
    print(f"\n📊 Evaluating {name} ({len(test_dataset)} samples)...")
    sources = []
    references = []
    inputs = []
    for item in test_dataset:
        src = " ".join(item['texts'])
        ref = item['summary']
        sources.append(src)
        references.append(ref)
        if processor:
            inputs.append(processor.create_guided_input(item['texts']))
        else:
            inputs.append(src)
    candidates = batch_generate(model, tokenizer, inputs, cfg, batch_size=4)

    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_values = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    bleu_values = []
    meteor_values = []
    coverage_values = []
    compression_values = []

    for src, ref, cand in zip(sources, references, candidates):
        if not cand:
            cand = ""
        r = rouge.score(ref, cand)
        rouge_values['rouge1'].append(r['rouge1'].fmeasure)
        rouge_values['rouge2'].append(r['rouge2'].fmeasure)
        rouge_values['rougeL'].append(r['rougeL'].fmeasure)
        bleu_values.append(compute_bleu(ref, cand))
        meteor_values.append(compute_meteor(ref, cand))
        cov, comp = compute_coverage_compression(src, cand, tokenizer)
        coverage_values.append(cov)
        compression_values.append(comp)

    try:
        P, R, F1 = bert_score_calculator(candidates, references, lang='en', model_type='distilbert-base-uncased', device=cfg.device, verbose=False)
        bert_values = (F1.cpu().numpy() * 100.0).tolist()
    except Exception:
        bert_values = [0.0] * len(candidates)

    return {
        'candidates': candidates,
        'references': references,
        'sources': sources,
        'rouge1': rouge_values['rouge1'],
        'rouge2': rouge_values['rouge2'],
        'rougeL': rouge_values['rougeL'],
        'bertscore_f1': bert_values,
        'bleu': bleu_values,
        'meteor': meteor_values,
        'coverage': coverage_values,
        'compression': compression_values
    }

# --------------------------
# Plotting utils (5 plots)
# --------------------------
def plot_training_loss(losses: List[float], save_path: Path):
    plt.figure(figsize=(8,4))
    plt.plot(losses, marker='o', linewidth=1)
    plt.title("Training Loss Curve")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

def plot_metric_bars(results_summary: Dict[str, Dict], save_path: Path):
    metrics = ['rouge1_mean','rouge2_mean','rougeL_mean','bertscore_mean']
    df = pd.DataFrame(results_summary).T
    df_plot = df[[m for m in df.columns if m in metrics]]
    df_plot.columns = ['ROUGE-1','ROUGE-2','ROUGE-L','BERTScore']
    ax = df_plot.plot(kind='bar', figsize=(11,5), rot=0, edgecolor='black', alpha=0.9)
    ax.set_ylabel('Score (%)')
    plt.title('Model Metric Comparison')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

def plot_rouge_violin(all_scores: Dict[str, Dict], save_path: Path):
    df = pd.DataFrame({k: np.array(v['rougeL']) * 100 for k, v in all_scores.items()})
    plt.figure(figsize=(10,5))
    sns.violinplot(data=df, inner='quartile')
    plt.title('ROUGE-L Distribution Across Models')
    plt.ylabel('ROUGE-L (%)')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

def plot_correlation_heatmap(all_scores: Dict[str, Dict], save_path: Path):
    rows = []
    for m, v in all_scores.items():
        rows.append({
            'model': m,
            'rouge1': np.mean(v['rouge1']) * 100,
            'rouge2': np.mean(v['rouge2']) * 100,
            'rougeL': np.mean(v['rougeL']) * 100,
            'bert': np.mean(v['bertscore_f1']),
            'bleu': np.mean(v['bleu']),
            'meteor': np.mean(v['meteor'])
        })
    df = pd.DataFrame(rows).set_index('model')
    corr = df.corr()
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='vlag', center=0)
    plt.title('Correlation Between Metrics (model-mean)')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

def plot_radar(results_summary: Dict[str, Dict], save_path: Path):
    labels = ['ROUGE-1','ROUGE-2','ROUGE-L','BERTScore']
    n = len(labels)
    angles = np.linspace(0, 2*np.pi, n, endpoint=False).tolist()
    angles += angles[:1]
    plt.figure(figsize=(7,7))
    ax = plt.subplot(polar=True)
    for model_name, vals in results_summary.items():
        stats = [vals['rouge1_mean'], vals['rouge2_mean'], vals['rougeL_mean'], vals['bertscore_mean']]
        stats += stats[:1]
        ax.plot(angles, stats, label=model_name)
        ax.fill(angles, stats, alpha=0.1)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    ax.set_title('Radar: Model Comparison')
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    plt.tight_layout()
    plt.savefig(save_path, dpi=150)
    plt.close()

# --------------------------
# Data loader helper: loads multiple datasets and samples according to ratios
# --------------------------
def load_and_mix_datasets(cfg: Config):
    """
    Attempts to load each dataset key in cfg.ratios.
    Key format:
      - either 'dataset_name' or 'dataset_name::config' (e.g., 'cnn_dailymail::3.0.0')
    Returns:
      train_list: list of dicts {'texts':[article], 'summary': highlights}
      test_list: same format
    """
    print("🔎 Loading datasets according to ratios:", cfg.ratios)
    # normalize ratios if some datasets fail to load
    available_parts = {}
    raw_datasets = {}

    for ds_key, frac in cfg.ratios.items():
        try:
            if '::' in ds_key:
                name, ver = ds_key.split('::', 1)
                ds_name, ds_config = name, ver
                ds = load_dataset(ds_name, ds_config, split='train', use_auth_token=False, streaming=False)
            else:
                ds_name = ds_key
                ds = load_dataset(ds_name, split='train', use_auth_token=False, streaming=False)
            raw_datasets[ds_key] = ds
            available_parts[ds_key] = frac
            print(f"  ✓ Loaded dataset: {ds_key} (size: {len(ds)})")
        except Exception as e:
            print(f"  ⚠️ Could not load {ds_key}: {e}. Skipping it.")
            continue

    if not raw_datasets:
        raise RuntimeError("No datasets could be loaded. Aborting.")

    # renormalize ratios
    total_frac = sum(available_parts.values())
    for k in list(available_parts.keys()):
        available_parts[k] = available_parts[k] / total_frac

    # compute counts for each dataset
    train_target = cfg.total_train_samples
    test_target = cfg.total_test_samples
    per_ds_train = {k: int(round(available_parts[k] * train_target)) for k in available_parts}
    per_ds_test = {k: int(round(available_parts[k] * test_target)) for k in available_parts}

    # collect samples
    train_samples = []
    test_samples = []

    for k, ds in raw_datasets.items():
        want_train = per_ds_train[k]
        want_test = per_ds_test[k]
        # map function: attempt to standardize fields
        def normalize(ex):
            # many datasets have different fields; try to find text and summary
            if 'article' in ex and 'highlights' in ex:
                return {'texts': [ex['article']], 'summary': ex['highlights']}
            # Multi-News has 'document' and 'summary' names
            if 'document' in ex and 'summary' in ex:
                return {'texts': [ex['document']], 'summary': ex['summary']}
            # WikiSum variants may have 'text' and 'summary' or 'article'/'summary'
            for tfield in ('text', 'article', 'document'):
                if tfield in ex and 'summary' in ex:
                    return {'texts': [ex[tfield]], 'summary': ex['summary']}
            # fallback: join values
            txt = " ".join(str(v) for v in ex.values() if isinstance(v, str))
            return {'texts': [txt], 'summary': ''}
        # convert to list for indexing
        ds_list = list(ds)
        random.shuffle(ds_list)

        # filter valid pairs
        filtered = []
        for ex in ds_list:
            ne = normalize(ex)
            if ne['texts'] and ne['summary']:
                filtered.append(ne)
        if not filtered:
            print(f"  ⚠️ No usable examples from {k} after normalization.")
            continue

        # sample desired counts (truncate if not enough)
        take_train = min(want_train, max(0, len(filtered) - want_test))
        take_test = min(want_test, len(filtered) - take_train)

        # take slices
        sampled_train = filtered[:take_train]
        sampled_test = filtered[take_train:take_train+take_test]

        # in rare case not enough for both, split proportionally
        if len(sampled_train) < want_train or len(sampled_test) < want_test:
            remain = filtered[take_train+take_test:]
            # fill train
            while len(sampled_train) < want_train and remain:
                sampled_train.append(remain.pop(0))
            # fill test
            while len(sampled_test) < want_test and remain:
                sampled_test.append(remain.pop(0))

        train_samples.extend(sampled_train)
        test_samples.extend(sampled_test)
        print(f"  → {k}: train {len(sampled_train)}, test {len(sampled_test)}")

    # If totals mismatch (rounding / missing), trim or pad from available
    if len(train_samples) > train_target:
        train_samples = train_samples[:train_target]
    elif len(train_samples) < train_target:
        # pad by re-sampling from available
        needed = train_target - len(train_samples)
        print(f"  ⚠️ Under-sampled train pool by {needed}. Resampling from available datasets.")
        pool = train_samples.copy() if train_samples else []
        # if empty, build from raw_datasets again
        for k, ds in raw_datasets.items():
            ds_list = list(ds)
            for ex in ds_list:
                train_samples.append(normalize(ex))
                if len(train_samples) >= train_target:
                    break
            if len(train_samples) >= train_target:
                break
        train_samples = train_samples[:train_target]

    if len(test_samples) > test_target:
        test_samples = test_samples[:test_target]
    elif len(test_samples) < test_target:
        needed = test_target - len(test_samples)
        print(f"  ⚠️ Under-sampled test pool by {needed}. Filling from train split (last samples).")
        # move some from end of train (small risk)
        move = min(needed, len(train_samples)//10)
        test_samples.extend(train_samples[-move:])
        test_samples = test_samples[:test_target]

    print(f"✓ Final pools: train {len(train_samples)} | test {len(test_samples)}")
    return train_samples, test_samples

# --------------------------
# Main pipeline
# --------------------------
def main():
    set_seed(config.seed)
    mkdir(config.output_dir)
    print("="*80)
    print("🚀 HTAS-V3 — Multi-dataset fine-tune (CNN + Multi-News + WikiSum mix)")
    print("="*80)
    print(f"Device: {config.device}")
    print(f"Train pool target: {config.total_train_samples}, Test pool target: {config.total_test_samples}")
    print(f"Dataset ratios: {config.ratios}")
    print("="*80)

    # Load and mix
    train_pool, test_pool = load_and_mix_datasets(config)

    # init tokenizer and processor
    tokenizer = AutoTokenizer.from_pretrained(config.htas_model_name)
    processor = HTASProcessor(config, tokenizer)

    # Build guided texts for train
    print("\n🔧 Creating HTAS-guided training inputs (this will compute SBERT embeddings & UMAP/HDBSCAN clustering for each item)...")
    guided_texts = []
    for item in tqdm(train_pool, desc="Processing train pool"):
        guided_texts.append(processor.create_guided_input(item['texts']))
    train_summaries = [item['summary'] for item in train_pool]
    train_dataset = SummarizationDataset(guided_texts, train_summaries, tokenizer, config)

    # Fine-tune HTAS model
    print("\n🚂 Fine-tuning HTAS model...")
    htas_model = AutoModelForSeq2SeqLM.from_pretrained(config.htas_model_name)
    htas_model, losses = train_model(htas_model, tokenizer, train_dataset, config)

    # Save HTAS model
    model_dir = config.output_dir / 'htas_model'
    mkdir(model_dir)
    htas_model.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)
    print(f"✓ HTAS model saved to {model_dir}")

    # Demo: short multi-doc
    print("\n🎯 Multi-doc demo (short):")
    demo_docs = [
        "World leaders gathered in Geneva for the 2025 Climate Summit, marking a crucial moment in global climate policy.",
        "Major economies announced historic climate investments at the Geneva summit, including EU and China actions.",
        "The summit's final agreement includes binding emissions targets, transparency mechanisms, and finance commitments."
    ]
    guided_demo = processor.create_guided_input(demo_docs)
    out_demo = batch_generate(htas_model, tokenizer, [guided_demo], config, batch_size=1)[0]
    print("Generated (demo):", out_demo[:800], "...")

    # Evaluate HTAS and baselines
    all_results = {}
    print("\n📊 Evaluating HTAS (fine-tuned)...")
    htas_eval = evaluate_model_full("HTAS-V3 (Fine-Tuned)", htas_model, tokenizer, test_pool, config, processor)
    all_results['HTAS-V3'] = htas_eval

    for mname in config.eval_models:
        try:
            print(f"\n📊 Loading baseline model {mname} (this may download large weights)...")
            tok = AutoTokenizer.from_pretrained(mname)
            mdl = AutoModelForSeq2SeqLM.from_pretrained(mname)
            baseline_eval = evaluate_model_full(mname, mdl, tok, test_pool, config, None)
            all_results[mname] = baseline_eval
            del mdl
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"⚠️ Could not evaluate {mname}: {e}")

    # Summarize numeric results
    results_summary = {}
    for name, res in all_results.items():
        results_summary[name] = {
            'rouge1_mean': float(np.mean(res['rouge1']) * 100), 'rouge1_std': float(np.std(res['rouge1']) * 100),
            'rouge2_mean': float(np.mean(res['rouge2']) * 100), 'rouge2_std': float(np.std(res['rouge2']) * 100),
            'rougeL_mean': float(np.mean(res['rougeL']) * 100), 'rougeL_std': float(np.std(res['rougeL']) * 100),
            'bertscore_mean': float(np.mean(res['bertscore_f1'])), 'bertscore_std': float(np.std(res['bertscore_f1'])),
            'bleu_mean': float(np.mean(res['bleu'])), 'bleu_std': float(np.std(res['bleu'])),
            'meteor_mean': float(np.mean(res['meteor'])), 'meteor_std': float(np.std(res['meteor'])),
            'coverage_mean': float(np.mean(res['coverage'])), 'compression_mean': float(np.mean(res['compression']))
        }

    # Save CSV of generated summaries
    rows = []
    for model_name, res in all_results.items():
        for i, cand in enumerate(res['candidates']):
            rows.append({'model': model_name, 'idx': i, 'candidate': cand, 'reference': res['references'][i]})
    df_out = pd.DataFrame(rows)
    df_out.to_csv(config.output_dir / 'generated_summaries.csv', index=False)

    # Save raw results (truncated for readability)
    safe_results = {}
    for k, v in all_results.items():
        safe_results[k] = {kk: (vv if not isinstance(vv, list) else vv[:50]) for kk, vv in v.items()}
    with open(config.output_dir / 'results_raw_sample.json', 'w') as f:
        json.dump(safe_results, f, indent=2)
    with open(config.output_dir / 'results_summary.json', 'w') as f:
        json.dump(results_summary, f, indent=2)

    # Print final table
    print("\n" + "="*80)
    print("📊 FINAL RESULTS SUMMARY")
    print("="*80)
    print("| Model | ROUGE-1 | ROUGE-2 | ROUGE-L | BERTScore | BLEU | METEOR | COVERAGE | COMPRESSION |")
    print("|-------|---------|---------|---------|-----------|------|--------|----------|-------------|")
    for name, vals in results_summary.items():
        print(f"| {name:<20} | {vals['rouge1_mean']:.2f}±{vals['rouge1_std']:.2f} | {vals['rouge2_mean']:.2f}±{vals['rouge2_std']:.2f} | {vals['rougeL_mean']:.2f}±{vals['rougeL_std']:.2f} | {vals['bertscore_mean']:.2f}±{vals['bertscore_std']:.2f} | {vals['bleu_mean']:.2f} | {vals['meteor_mean']:.2f} | {vals['coverage_mean']:.2f} | {vals['compression_mean']:.2f} |")

    # Generate plots
    print("\n📈 Generating plots...")
    mkdir(config.output_dir / 'plots')
    try:
        plot_training_loss(losses, config.output_dir / 'plots' / 'training_loss.png')
    except Exception:
        print("⚠️ Could not plot training loss (missing 'losses').")
    plot_metric_bars(results_summary, config.output_dir / 'plots' / 'metric_comparison.png')
    plot_rouge_violin(all_results, config.output_dir / 'plots' / 'rouge_violin.png')
    plot_correlation_heatmap(all_results, config.output_dir / 'plots' / 'metric_correlation.png')
    plot_radar(results_summary, config.output_dir / 'plots' / 'radar_comparison.png')

    print(f"\n✓ Saved plots to {config.output_dir / 'plots'}")
    print(f"✓ Saved generated summaries to {config.output_dir / 'generated_summaries.csv'}")
    print(f"✓ Saved results JSONs to {config.output_dir}")

    # Cleanup
    try:
        del htas_model
    except Exception:
        pass
    gc.collect()
    torch.cuda.empty_cache()
    print("\n🎉 Pipeline complete. All outputs are in:", config.output_dir)

# --------------------------
# Run
# --------------------------
if __name__ == "__main__":
    main()


2025-10-29 17:02:43.899691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761757364.116632      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761757364.185377      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🚀 HTAS-V3 — Multi-dataset fine-tune (CNN + Multi-News + WikiSum mix)
Device: cuda
Train pool target: 8000, Test pool target: 2000
Dataset ratios: {'cnn_dailymail::3.0.0': 0.35, 'multi_news': 0.35, 'xsum': 0.3}
🔎 Loading datasets according to ratios: {'cnn_dailymail::3.0.0': 0.35, 'multi_news': 0.35, 'xsum': 0.3}


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

...

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]


📊 Evaluating google/pegasus-cnn_dailymail (2000 samples)...



📊 FINAL RESULTS SUMMARY
| Model | ROUGE-1 | ROUGE-2 | ROUGE-L | BERTScore | BLEU | METEOR | COVERAGE | COMPRESSION |
|-------|---------|---------|---------|-----------|------|--------|----------|-------------|
| HTAS-V3              | 35.58±11.17 | 12.80±10.09 | 22.73±9.77 | 79.70±3.91 | 28.44 | 0.00 | 11.12 | 15.85 |
| facebook/bart-large-cnn | 29.46±14.59 | 11.13±12.72 | 20.13±12.80 | 78.43±4.83 | 24.34 | 0.00 | 10.19 | 22.99 |
| t5-base              | 27.18±12.61 | 9.24±10.25 | 18.04±10.09 | 77.26±4.68 | 21.51 | 0.00 | 12.44 | 19.36 |
| google/pegasus-cnn_dailymail | 32.06±16.96 | 14.06±16.14 | 23.01±16.47 | 78.06±5.53 | 26.16 | 0.00 | 11.29 | 20.25 |

📈 Generating plots...

✓ Saved plots to /content/htas_output/plots
✓ Saved generated summaries to /content/htas_output/generated_summaries.csv
✓ Saved results JSONs to /content/htas_output

🎉 Pipeline complete. All outputs are in: /content/htas_output
